In [7]:

import os
os.environ['P8_PG_HOST'] = 'localhost'
os.environ['P8_PG_PORT'] = '25432'
os.environ['P8_PG_PASSWORD'] = os.environ.get('P8_TEST_BEARER_TOKEN')
import nest_asyncio
nest_asyncio.apply()

In [8]:
import asyncio
import httpx
import os
import json
#print(os.environ.get('P8_TEST_BEARER_TOKEN'))

async def stream_openai_endpoint(url: str):
    headers = {
        "Authorization": f"Bearer {os.environ.get('P8_TEST_BEARER_TOKEN')}",
        "Accept": "text/event-stream",
        "Content-Type": "application/json"
    }

    payload = {
        "model": 'gpt-4.1-mini',
        "messages": [
           # {"role": "user", "content": "What can you tell me about Resonance"}
            {"role": "user", "content": "What did i just ask about?"}
        ],
        "stream": True
    }
        
    async with httpx.AsyncClient(timeout=None) as client:
        async with client.stream("POST", url, headers=headers, json=payload) as response:
            async for line in response.aiter_lines():
                #print(line)
                if line.startswith("data: "):
                    data = line[len("data: "):]
                    if data.strip() == "[DONE]":
                        #print("[DONE]")
                        break
                    a = json.loads(data)['choices'][0]['delta']['content']
                    print(a, end='')

# Example usage:
await stream_openai_endpoint("http://localhost:5009/v1/agents/executive-ExecutiveDocs/chat/completions?user_id=4114f279-f345-511b-b375-1953089e078f")


You just asked: "What did i just ask about?"

There are no previous questions or context in this chat thread, so there is nothing you asked about before this. If you have a specific topic or question in mind, please let me know!

In [9]:
from percolate.models import AbstractModel
import percolate as p8
 

In [10]:
from percolate.interface import try_load_model


In [11]:
M = p8.try_load_model('executive.ExecutiveDocs')

In [32]:
from IPython.display import Markdown
#Markdown(p8.Agent(M)("If i wanted to scale REsonance in 2026, what do you think, based on the model, would be a good strategy for breaking into new markets"))
#Markdown(p8.Agent(M)("tell me about the network effects that REsonance can levergage to scale different parts of the business"))
Markdown(p8.Agent(M)("If i were a brand, how would i launch with the REsonance platform. Please provide easy to follow steps"))

2025-05-28 20:34:59.383 | DEBUG    | percolate.services.FunctionManager:add_function:40 - added function help
2025-05-28 20:34:59.644 | DEBUG    | percolate.services.FunctionManager:add_function:40 - added function get_entities
2025-05-28 20:34:59.647 | DEBUG    | percolate.services.FunctionManager:add_function:40 - added function search
2025-05-28 20:34:59.648 | DEBUG    | percolate.services.FunctionManager:add_function:40 - added function activate_functions_by_name
2025-05-28 20:34:59.649 | INFO     | percolate.services.ModelRunner:__init__:57 - ******Constructed agent executive.ExecutiveDocs******
2025-05-28 20:35:00.597 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:320 - invoking model gpt-4.1, is_streaming=False
2025-05-28 20:35:02.273 | DEBUG    | percolate.services.llm.LanguageModel:__call__:198 - response=<Response [200]>, context=CallingContext(session_id=None, session_context=None, prefer_json=False, response_model=None, username=None, user_id=None, channe

Here are easy-to-follow steps for launching your brand with the Resonance platform:

1. Define Your Brand Intent
   - Clearly articulate your brand’s mission, values, and target audience.
   - Express your brand intent so it can be captured and translated into product and experience.

2. Design Your Products
   - Use Resonance’s create.ONE platform to design your garments.
   - Leverage design support (patternmaking, tech packs, etc.) if needed.

3. Set Up Your Online Store
   - Organize your styles and products for sale.
   - Resonance helps you set up an online store or connect to retailers.

4. Sell Before You Make
   - Launch your products for sale—no need to pre-make inventory.
   - Accept orders from customers in real time.

5. On-Demand Manufacturing
   - Once an order is placed, Resonance’s platform coordinates with material and sewing factories to make each garment on demand.
   - This eliminates waste and reduces costs.

6. Fulfillment and Delivery
   - Finished garments are shipped directly to your customers.
   - Resonance manages logistics and fulfillment.

7. Optimize and Iterate
   - Use real-time sales and production data to optimize your designs, pricing, and marketing.
   - Quickly react to trends and customer feedback.

8. Scale Sustainably
   - As your brand grows, continue to add new styles and products.
   - Benefit from Resonance’s integrated, sustainable, and profitable value chain.

Key Benefits:
- No inventory risk or overproduction.
- Fast reaction to market trends.
- Transparent, sustainable, and profitable operations.

If you need more detail on any step, let me know!

In [27]:

p8.repository(M).search("Saoirse Amarteifio")

[{'query_text': None,
  'confidence': None,
  'relational_result': None,
  'vector_result': [{'id': '2bdea8b6-6508-58de-8339-79db026c92e0',
    'uri': 's3://res-data-platform/box-sync/v1/231707137377/4_Labor_and_Employment/Agreements/Resonance_Companies_Inc/Active_Full-Time_Employees/Andrew_Erskine_-_Offer_Letter_Executed.pdf',
    'name': 'Andrew Erskine - Offer Letter Executed (chunk 27)',
    'userid': None,
    'content': '"Saoirse Amarteifio"\n\n- "Director, ONE Platform Engineering"\n\n- "Confidentiality, Intellectual Property, and Non-Solicitation Agreement Page 5"\n\n- "Doc ID: 2cba0941367e4982370574810cbb7595773e719"\n\n2. **Tables, Charts, or Structured Data:**\n\n- There are no tables, charts, or structured data present on the page.\n\n3. **Images, Diagrams, or Visual Elements:**\n\n- There are signatures on the page. The signature line for "By:" is filled with a stylized signature. There\'\'s also a stylized signature on the line for "Andrew Erskine".\n\n4. **Layout and For

In [ ]:
import sys
from warnings import filterwarnings
filterwarnings('ignore')
from percolate.models.p8.types import Resources

sys.path.append('/Users/sirsh/code/res/one-core')

In [ ]:
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/INST_018.wav",
    parsing_mode="extended",
    chunk_size=1000,
    chunk_overlap=200,
    save_to_db=False,
    
)

len(resources)

In [ ]:

# Simple chunking - basic text extraction (fast, no API costs)
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/6-Month Roadmap & Milestones.pdf",
    parsing_mode="simple",
    chunk_size=1000,
    chunk_overlap=200,
    save_to_db=False
)

len(resources)

In [ ]:
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/Transit Insurance Form.pdf", 
    parsing_mode="extended",
    save_to_db=True
)
resources

In [ ]:
resources = Resources.chunked_resource(
    uri="/path/to/document.pdf", 
    parsing_mode="extended",
    chunk_size=1000,
    chunk_overlap=200
)

In [ ]:
"""integration test"""

In [ ]:
from one.services.ask_one import AgentCallingContext
from one.services.slack.models import SlackMessage
from percolate.utils import make_uuid
def _get_userid_from_username(name:str):
    """map the user id from the email address or slack name"""
    from percolate.services import PostgresService
    pg = PostgresService()
    Q = f"""
            SELECT * FROM p8."User" where LOWER(email) = LOWER(%s) or LOWER(slack_id) = LOWER(%s) limit 1
    """
    
    data = pg.execute(Q, data=(name, name))
    if data:
        return data[0]['id']
    
    
channel_ts = f"123445"
message = SlackMessage(user='U01JDKSB196', 
                       text='hi, can you tell me thre things about resonance', 
                       thread_ts="123454", 
                       ts=channel_ts, 
                       channel='C08TE6NNY7L')
"""TODO think about this session id interface"""
ctx = AgentCallingContext(
    channel_context=message.channel,
    streaming_callback=None,#,buffer.add_data,
    username=message.user,

    session_id=make_uuid( message.thread_ts or message.ts), #the slack message thread is used as a session id - when there is no message thread_ts its because we are the originator
    channel_ts=str(channel_ts),
    session_context=f"{message.channel}-{message.thread_ts or message.ts}",
    s3_files=None,
    process_files_async=True,
    user_id=_get_userid_from_username(message.user)
    
)
setattr(ctx,'max_iterations',5)

In [ ]:
import percolate as p8
from percolate.models import PercolateAgent

p8.Agent(PercolateAgent).run('what is a fashion brand',context=ctx)

In [ ]:
#!pip install -U boto3

In [ ]:
from percolate.services import S3Service

s =S3Service()
s

In [ ]:
from io import BytesIO
with open("/Users/sirsh/Downloads/6_Proposed Endeavor Guide and Clarifications.pdf", 'rb') as f:
    data = BytesIO(f.read())
    data.seek(0)
    s.upload_filebytes_to_uri("s3://percolate/test/file.pdf", data)

In [ ]:
s.upload_file_to_uri("s3://percolate/test/file.pdf",
                     "/Users/sirsh/Downloads/6_Proposed Endeavor Guide and Clarifications.pdf" )

In [ ]:
import percolate as p8
from percolate.models.media.tus import register_tus_models
from percolate.models import User
#p8.repository(User).register()
#register_tus_models()
from percolate.services import PostgresService

pg = PostgresService()
 

In [ ]:
pg.execute(""" select * from p8."Resources" order by updated_at desc limit 5 """)

In [ ]:
import requests

session = requests.Session()
session.cookies.set('session', 'eyJzeW5jX2ZpbGVzIjogZmFsc2UsICJ0b2tlbiI6IHsiYWNjZXNzX3Rva2VuIjogInlhMjkuYTBBVzRYdHhocFU5Q1ZwU284R2FwR0JQaHZYcHA0S1BJdGREbW9ocmc3ZFNBbllXdEN4ajM3N2pRcTA1Sk03UnhsR3phSk05cWNkU09SWk5rdjlxckdNV1NfUkVqRFhGMVU1Z3FtN0ZDbEhUb3RUM1N3SldyUmJCZGd3d2N2SWlKbWZmSzZDSWZDemdYNHdJZTQ0ZDFlVFV2emkzc1Y4Z2FTTkstX2tENDVhQ2dZS0FXUVNBUmNTRlFIR1gyTWlHLXRJNXlmYzBFTTA5MnFTMnN6aFpnMDE3NSIsICJleHBpcmVzX2luIjogMzU5OSwgInJlZnJlc2hfdG9rZW4iOiAiMS8vMDVZWDNsZG04WXVJeUNnWUlBUkFBR0FVU053Ri1MOUlyYW1wNHlKR2Y5TnA0UW9aUlVGYkdQeVJXaTlWRHZUX0VLdVJ0cFJ3YnJUcHYxS1FVY1VtOFlBV1hQWURBNnhkckc0TSIsICJzY29wZSI6ICJodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL2RyaXZlLnJlYWRvbmx5IGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvdXNlcmluZm8ucHJvZmlsZSBodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL3VzZXJpbmZvLmVtYWlsIGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvZG9jdW1lbnRzLnJlYWRvbmx5IGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvZ21haWwucmVhZG9ubHkgb3BlbmlkIiwgInRva2VuX3R5cGUiOiAiQmVhcmVyIiwgImlkX3Rva2VuIjogImV5SmhiR2NpT2lKU1V6STFOaUlzSW10cFpDSTZJalkyTUdWbU0ySTVOemcwWW1SbU5UWmxZbVU0TlRsbU5UYzNaamRtWWpKbE9HTXhZMlZtWm1JaUxDSjBlWEFpT2lKS1YxUWlmUS5leUpwYzNNaU9pSm9kSFJ3Y3pvdkwyRmpZMjkxYm5SekxtZHZiMmRzWlM1amIyMGlMQ0poZW5BaU9pSXhNRGs1TWpNM09EazBPVE0xTFhBemJqZG5hMncyY3prM2FHb3lNRGh1YjNGdE5ISm9PSEpyTTJkaVluWjBMbUZ3Y0hNdVoyOXZaMnhsZFhObGNtTnZiblJsYm5RdVkyOXRJaXdpWVhWa0lqb2lNVEE1T1RJek56ZzVORGt6TlMxd00yNDNaMnRzTm5NNU4yaHFNakE0Ym05eGJUUnlhRGh5YXpOblltSjJkQzVoY0hCekxtZHZiMmRzWlhWelpYSmpiMjUwWlc1MExtTnZiU0lzSW5OMVlpSTZJakV3TWpFMk5UUTNOakV4TkRnd05EQTVOakl6TUNJc0ltVnRZV2xzSWpvaVlXMWhjblJsZVVCbmJXRnBiQzVqYjIwaUxDSmxiV0ZwYkY5MlpYSnBabWxsWkNJNmRISjFaU3dpWVhSZmFHRnphQ0k2SW5OclYyWktjbTVsTlRaMmEyaFRMVnBKVEhOSlNXY2lMQ0p1YjI1alpTSTZJbkp2WlRNM1UzWkJjbmMyU0VwSVVsQTFTbTl0SWl3aWJtRnRaU0k2SWxOaGIybHljMlVnUVcxaGNuUmxhV1pwYnlJc0luQnBZM1IxY21VaU9pSm9kSFJ3Y3pvdkwyeG9NeTVuYjI5bmJHVjFjMlZ5WTI5dWRHVnVkQzVqYjIwdllTOUJRMmM0YjJOTWFWbFFNM0J0TFhSNmFITldURkp1T1ZoaVoxQlFWa2g0U0RCYWVUQmpSMmROZERKcVlVVkJZMnR3VDBsdlRGUkVaejF6T1RZdFl5SXNJbWRwZG1WdVgyNWhiV1VpT2lKVFlXOXBjbk5sSWl3aVptRnRhV3g1WDI1aGJXVWlPaUpCYldGeWRHVnBabWx2SWl3aWFXRjBJam94TnpRM05EQTJPVGszTENKbGVIQWlPakUzTkRjME1UQTFPVGQ5LmM5NWQ5WGtWREFQcjdjU3hBUWVtZzFYWXROSjREcHJhS2JIYlUwb0t1VmNlcDdybTNxQnRqOTZ4c0VnSWc5SXBWVlQ5Ym4zTDF2alU5OTYyVDdQeC1BWEdNYzRab25YX3FYTGc2M1pueGhBVHNIcXdoQ2VoS2N5NVZxYkxkbVVtYnlsZmFYR0pIbFp5ZDhRbkU1RVppY3B1d3ZhN1U5Y3F2Q184ZTVERkN6Ymk5d0V2bHNmc21Vb3U3U0dWdERkX2dqVUxfNTY4RldhSktrNmt3aUhBZXRUaEQ5WmZUaGgybE1vZHdrVHhuOXVzTWhNZ2oyeENvLXR4bjdqa3hRNnk1NGpINE1DRnhUZzk4V2lRdFJvWkxUbm9neXVYbGZzVkI3V3FQdllqTjlrR2tmMnU3SzJCOHFuMkJGT1NjTFJPaHRGSzJDcTIzVm1WZzhNMmVXQlB6dyIsICJyZWZyZXNoX3Rva2VuX2V4cGlyZXNfaW4iOiA2MDQ3OTksICJleHBpcmVzX2F0IjogMTc0NzQxMDU5NiwgInVzZXJpbmZvIjogeyJpc3MiOiAiaHR0cHM6Ly9hY2NvdW50cy5nb29nbGUuY29tIiwgImF6cCI6ICIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwgImF1ZCI6ICIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwgInN1YiI6ICIxMDIxNjU0NzYxMTQ4MDQwOTYyMzAiLCAiZW1haWwiOiAiYW1hcnRleUBnbWFpbC5jb20iLCAiZW1haWxfdmVyaWZpZWQiOiB0cnVlLCAiYXRfaGFzaCI6ICJza1dmSnJuZTU2dmtoUy1aSUxzSUlnIiwgIm5vbmNlIjogInJvZTM3U3ZBcnc2SEpIUlA1Sm9tIiwgIm5hbWUiOiAiU2FvaXJzZSBBbWFydGVpZmlvIiwgInBpY3R1cmUiOiAiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EvQUNnOG9jTGlZUDNwbS10emhzVkxSbjlYYmdQUFZIeEgwWnkwY0dnTXQyamFFQWNrcE9Jb0xURGc9czk2LWMiLCAiZ2l2ZW5fbmFtZSI6ICJTYW9pcnNlIiwgImZhbWlseV9uYW1lIjogIkFtYXJ0ZWlmaW8iLCAiaWF0IjogMTc0NzQwNjk5NywgImV4cCI6IDE3NDc0MTA1OTd9fSwgInNlc3Npb25faWQiOiAiYzU4ZmQ1MWYtNDAzZC00MjU3LTllMTgtMTc3MzdlNDRmZDNmIn0=.aCdQlQ.qHKMWsxfHuBLHnk3Ck3Ux_tiqp8')

response = session.get('http://localhost:5008/auth/ping')
print(response.json())

response = session.get('http://localhost:5008/auth/session/info')
print(response.json())

In [ ]:
session.get('http://localhost:5008/tus/user/uploads?query=anything').json()

In [ ]:
response = session.get('http://localhost:5008/auth/me')
print(response.json())

In [ ]:
from percolate.api.routes.auth.utils import get_user_from_email, is_valid_token_for_user, extract_user_info_from_token
extract_user_info_from_token('eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6ImlBS2ZoVXRMU1ZDbkR4aVE1X3NwUUEiLCJub25jZSI6IjhGamhCTE5TYzBFeDJ2Qm5KSVA1IiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTk4Mjk0LCJleHAiOjE3NDc2MDE4OTR9.P0goYoml9dxdPHiH-f7IjeEFWMv6qDT6cv-GxHma2drk14pkIuwwdU0w2OCDSlFRGc4hfePIvMU3KIddNPGnkc8qvBKDNvafdBkndMQ0D1rae8BNScVr3EuZiikv_O54Eff1Lv6cmcKd978m5I7p8xVx-2aGGrxGELz3OFszgnh6POo1efiuznp6sPIQJbskfI1a8YR1_GArU9jkwhjhczEsibWNtLn5hIzox02qBTdjfrh69eOOKSmysb2SjWl54AmWxubYfR1M1IO0CAPHUQFzVrdouUsBvZnIk3xWnF-roSMrt68aoWWCtzAxtupzy3ziY8RdTVyWy7uEnr-k6g')

In [ ]:
import requests

session = requests.Session()
#session.cookies.set('session', 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6InNlN2FuOVhHd19BeEVGdkFOSTZSckEiLCJub25jZSI6InZWRlVBekpTU3RsMGZmQnp3RXBsIiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTkzNjk4LCJleHAiOjE3NDc1OTcyOTh9.Z61fPlHZ__DZ38rhlU-PmloMjfs6Tj2xLFqgLj5r2yOHsPDUSEmBYm8Ay7qEmUj5kWV-iyHRgW1c7Z-p3z1k8KUyPVsKXXbZlHvlHUDBmTQo8Uy3axHyei02Vz2ejImBspjlLzjg1bf_wQUXOo5pAdR8D0elaWoRLseslRvyLdZf3k2mk_pxy3c3utkefco1psIaGcdRC44OIdqCCWtPh1MXsSu0fMW4vsdpJQYRrIlmodMWwTLph6Rv2jJ6Rl5JQU3OG6Bx9dW5XQld1k1eQuP2_UEij4-lo3_JsJD4zdbFDSAbiNUbZICof6tatbsDiCpHd3o9-AV_BBlcn_lKdA')

session.cookies.set('session', 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6Im5KcnJIMm9lanZvSHRrSUJyS1FLckEiLCJub25jZSI6Imc1bVk5Z3AyaVduSDF6Q2U2MFl5IiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTk4NTcwLCJleHAiOjE3NDc2MDIxNzB9.NpUvgWfPSpQLaQL8UR1mr2Vd4S7-3ZLEoeK2Dbpbi_uBES51dm8MUFnDTQPNULwo-IVUE9GBAr5HInqTx6L95cTiAohTUG-Qzio96rwtnNYF0YWkBByhTKWRKhdQ6YK97TwGffZMrEzaQ_QvHe_XF17exJpKkRLtjijCuMmmAMMqbdgtsd5XVtxSma_HFapIi4dTcZOiVqWglzsGCXWxvZqtWXiHii1niKFYb95aF8q4sRgsRGs4Z7cCc1l6YzD8sSnkTPyay_qWuAWvMlS3iY9x3777zG82U9Fi38sZ2IaQ7CS1IbZ7CrBo45-2iuh28PSw7H0v2qGcTwfOF0lhTA')

response = session.get('https://eepis.percolationlabs.ai/auth/ping')
print(response.json())

response = session.get('https://eepis.percolationlabs.ai/auth/session/info')
print(response.json())

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?limit=10').json()

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?query=CevolvingAnyibodies.').json()

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?query=Eta Carinae pulsed in the distance.').json()